# Linear Program
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eastspring-investments/herauni/blob/main/notebooks/linear_programming.ipynb)

## Google Colab Settings

In [74]:
try:
  import google.colab
  COLAB = True
except:
  COLAB = False


if COLAB:
    ! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
    ! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
    import sys
    sys.path.append('/usr/local/lib/python3.7/site-packages/')

    ! conda install -c conda-forge pyomo -y
    ! conda install -c conda-forge pyomo.extras -y
    ! conda install -c conda-forge coincbc -y
    ! conda install -c conda-forge ipopt -y
    ! conda install -c conda-forge glpk -y

## Imports

In [75]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

## Basic LP Problem: Example 1

### Problem statement
- **We are holding a corporate pizza & prosecco party in a division and we need to optimise how many pizza and prosecco to buy**
    - We have 20 people
    - Each **pizza** costs \$30
    - Each **prosecco** costs \$50
    - CONSTRAINTS
        - Every staff drinks at least half a bottle of prosecco
        - Every staff eats at least 2 slices of pizza

### Step 1: Define Decision Variables
- DEFINE HOW MANY PIZZA & PROSECCO TO BUY

In [91]:
# Instantiate model and assign to model object
model = ConcreteModel()

In [92]:
# Define variables
model.n_pizza = Var(domain=NonNegativeIntegers)
model.n_prosecco = Var(domain=NonNegativeIntegers)

### Step 2: Define Objective Function
- 30 * number pizza + 50 * number of prosecco

In [93]:
# Number of people 
n_people = 20

In [94]:
# Unit costs
unit_costs = {
    'pizza': 30,
    'prosecco': 50
}

In [95]:
# Objective function 
model.cost = Objective(expr=unit_costs['pizza']*model.n_pizza + \
                       unit_costs['prosecco']*model.n_prosecco)

### Step 3: Define Constraints

In [96]:
# Contraint 1: each person drinks at least half a bottle of prosecco 
model.prosecco_demand = Constraint(expr = model.n_prosecco >= 0.5 * n_people)

# Constraint 2: each person eats at least 2 slices of pizza
# Each has 6 slices
model.pizza_demand = Constraint(expr = model.n_pizza >= (2/6) * n_people)

### Step 4: Solve 

In [97]:
# Solve linear model
# Currenly using GPLK
# Other powerful commercial optimisers can be used like Gurobi or CPLEX
results = SolverFactory('glpk').solve(model)

results.write()
if results.solver.status == 'ok':
    model.pprint()


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 710.0
  Upper bound: 710.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 3
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.006757259368896484
# ----------------------------------------------------------
#   Solution Information
# ------------------------------------

### Step 5: Analyse

In [98]:
# Display solution
print('\nCost = ', model.cost())

print('\nDecision Variables')
print('Number of Pizza = ', model.n_pizza())
print('Number of Prosecco = ', model.n_prosecco())

print('\nConstraints')
#print('Pizza Demand  = ', model.pizza_demand())
#print('Prosecco Demand = ', model.prosecco_demand())


Cost =  710.0

Decision Variables
Number of Pizza =  7.0
Number of Prosecco =  10.0

Constraints


## Basic LP Problem: Example 2
- Let's build a roboadvisor to optimise for portfolio allocation!

| Fund Name  | Risk Level | Category               | Average Annual Return (ARR) |
|------------|------------|------------------------|-----------------------------|
| Fund One   | 1          | Money market fund      | 4.5                         |
| Fund Two   | 2          | Money market fund      | 5.62                        |
| Fund Three | 2          | Bond fund              | 6.8                         |
| Fund Four  | 3          | High yield bond fund   | 10.15                       |
| Fund Five  | 5          | Aggressive growth fund | 20.6                        |

- Risk scale of 1 (very conservative) to 5 (very risky). 
- We have 10,000 USD
- Maximise the average annual return on investment, subject to the following restrictions:
    - The average risk level of the entire investment should not exceed 2.5
    - At least 30% of the investment should be placed in money market funds
    - At most 2,000 USD should be invested in the aggressive growth fund.

### Step 1: Define Decision Variables

In [100]:
# Instantiate model and assign to model object
model = ConcreteModel()

In [101]:
# Define variables
model.allocation_fund_1 = Var(domain=NonNegativeReals)
model.allocation_fund_2 = Var(domain=NonNegativeReals)
model.allocation_fund_3 = Var(domain=NonNegativeReals)
model.allocation_fund_4 = Var(domain=NonNegativeReals)
model.allocation_fund_5 = Var(domain=NonNegativeReals)

### Step 2: Define Objective Function
- TRYING TO MAXIMIZE THE RETURN OF OUR 10,000 USD

In [102]:
# Objective function 
model.cost = Objective(expr=(4.5 * model.allocation_fund_1 + \
                       5.62 * model.allocation_fund_2 + \
                       6.8 * model.allocation_fund_3 + \
                       10.15 * model.allocation_fund_4 + \
                       20.6 * model.allocation_fund_5)/100, sense=maximize)

### Step 3: Define Constraints

In [103]:
# Total cash allocation
model.total_cash = Constraint(expr=model.allocation_fund_1 + \
                              model.allocation_fund_2 + \
                              model.allocation_fund_3 + \
                              model.allocation_fund_4 + \
                              model.allocation_fund_5 <= 10000
                             )

In [104]:
# Mean Risk
# The average risk level of the entire investment should not exceed 2.5
model.mean_risk = Constraint(expr=(model.allocation_fund_1 + \
                              2*model.allocation_fund_2 + \
                              2*model.allocation_fund_3 + \
                              3*model.allocation_fund_4 + \
                              5*model.allocation_fund_5) / 10000 <= 2.5)

In [105]:
# Money Market Exposure
# At least 30% of the investment should be placed in money market funds
model.mm_exposure = Constraint(expr=model.allocation_fund_1 + \
                              model.allocation_fund_2 >= 3000)

In [106]:
# Aggressive Growth Exposure
# At most 2,000 USD should be invested in the aggressive growth fund.
model.ag_exposure = Constraint(expr=model.allocation_fund_5 <= 2000)

### Step 4: Solve 

In [107]:
# Solve linear model
# Currenly using GPLK
# Other powerful commercial optimisers can be used like Gurobi or CPLEX
results = SolverFactory('glpk').solve(model)

results.write()
if results.solver.status == 'ok':
    model.pprint()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 969.75
  Upper bound: 969.75
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 6
  Number of nonzeros: 14
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.007001399993896484
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [108]:
# Display solution
print('\nReturn = ', model.cost())

print('\nDecision Variables')
print('Allocation to fund 1', model.allocation_fund_1())
print('Allocation to fund 2', model.allocation_fund_2())
print('Allocation to fund 3', model.allocation_fund_3())
print('Allocation to fund 4', model.allocation_fund_4())
print('Allocation to fund 5', model.allocation_fund_5())

print('-'*50)

print('\nConstraints')
print('Total cash', model.total_cash())
print('Mean risk', model.mean_risk())
print('Money market exposure', model.mm_exposure())
print('Aggressive growth exposure', model.ag_exposure())


Return =  969.75

Decision Variables
Allocation to fund 1 4500.0
Allocation to fund 2 0.0
Allocation to fund 3 0.0
Allocation to fund 4 3500.0
Allocation to fund 5 2000.0
--------------------------------------------------

Constraints
Total cash 10000.0
Mean risk 2.5
Money market exposure 4500.0
Aggressive growth exposure 2000.0


## Basic LP Problem: Example 3 (Practice)

### Problem Statement
- The following is purely hypothetical. 
- **We have quant developers in 10 countries and we want to have 24 hours trading coverage**
    - Original QD count
        - Singapore: 4
        - London: 4
        - New York: 4
        - Vietnam: 3
        - India: 3
        - Hong Kong: 2
        - Indonesia: 2
        - Malaysia: 2
        - Taiwan: 2
        - Korea: 3
    - Asia coverage: we must have at least 6 quant developers at any point in time
        - Only Singapore, Vietnam, India, Hong Kong, Indonesia, Malaysia, Taiwan and Korea can handle Asia time zone
    - UK coverage: we must have at least 4 quant developers at any point in time
        - Only London and New York can handle UK time zone
        - Singapore, Hong Kong, India and Vietnam can handle UK time zone at 20% extra compensation
    - US coverage: we must have at least 4 quant developers at any point in time
        - Only London and New York can handle US time zone
        - Singapore, Hong Kong and India can handle US time zone at 30% extra compensation
    - New QD count: after leave/medical/resignations
        - Singapore: 1
        - London: 1
        - New York: 1
        - Vietnam: 3
        - India: 3
        - Hong Kong: 2
        - Indonesia: 2
        - Malaysia: 2
        - Taiwan: 2
        - Korea: 3